In [1]:
import os
import sys
import pandas as pd
import requests
from collections import defaultdict

In [2]:
# Check if running in the GitHub Actions environment
if 'GITHUB_ACTIONS' in os.environ:
    project_path = os.getcwd()
else:
    # Assuming your script is in the 'scripts' directory
    project_path = os.path.abspath(os.path.join(os.getcwd(), '../../..'))

# Add the project directory to the PYTHONPATH if it's not already there
if project_path not in sys.path:
    sys.path.append(project_path)

# Now you can import your custom module
from data_utils.data_processing import download_file, process_zip_file


In [3]:
from data_utils.data_processing import download_file, read_csv_file


In [4]:
def get_data_paths():
    # Check if running in the GitHub Actions environment
    if 'GITHUB_ACTIONS' in os.environ:
        base_path = os.path.join(os.getcwd(), 'data')
    else:
        # Assuming your script is in the 'scripts' directory
        base_path = os.path.abspath(os.path.join(os.getcwd(), '../../../data'))

    source_path = os.path.join(base_path, "source/ine/empleo")
    processed_path = os.path.join(base_path, "processed/ine/empleo")
    
    return source_path, processed_path


In [5]:
source_dir, processed_dir = get_data_paths()

print(f"Source dir: {source_dir}")
print(f"Processed dir: {processed_dir}")


Source dir: /Users/ernestolaval/Documents/Github Repositories/data_chile/data/source/ine/empleo
Processed dir: /Users/ernestolaval/Documents/Github Repositories/data_chile/data/processed/ine/empleo


In [6]:
os.makedirs(source_dir, exist_ok=True)
os.makedirs(processed_dir, exist_ok=True)


In [7]:
# List of file URLs
file_urls = [
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2024/csv/ene-2024-04-mam.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2024/csv/ene-2024-03-fma.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2024/csv/ene-2024-02-efm.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2024/csv/ene-2024-01-def.csv",

    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2023/csv/ene-2023-12-nde.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2023/csv/ene-2023-11-ond.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2023/csv/ene-2023-10-son.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2023/csv/ene-2023-09-aso.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2023/csv/ene-2023-08-jas.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2023/csv/ene-2023-07-jja.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2023/csv/ene-2023-06-mjj.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2023/csv/ene-2023-05-amj.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2023/csv/ene-2023-04-mam.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2023/csv/ene-2023-03-fma.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2023/csv/ene-2023-02-efm.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2023/csv/ene-2023-01-def.csv",

    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2022/csv/ene-2022-12-nde.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2022/csv/ene-2022-11-ond.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2022/csv/ene-2022-10-son.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2022/csv/ene-2022-09-aso.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2022/csv/ene-2022-08-jas.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2022/csv/ene-2022-07-jja.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2022/csv/ene-2022-06-mjj.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2022/csv/ene-2022-05-amj.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2022/csv/ene-2022-04-mam.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2022/csv/ene-2022-03-fma.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2022/csv/ene-2022-02-efm.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2022/csv/ene-2022-01-def.csv",

    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2021/csv/ene-2021-12-nde.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2021/csv/ene-2021-11-ond.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2021/csv/ene-2021-10-son.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2021/csv/ene-2021-09-aso.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2021/csv/ene-2021-08-jas.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2021/csv/ene-2021-07-jja.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2021/csv/ene-2021-06-mjj.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2021/csv/ene-2021-05-amj.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2021/csv/ene-2021-04-mam.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2021/csv/ene-2021-03-fma.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2021/csv/ene-2021-02-efm.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2021/csv/ene-2021-01-def.csv",

    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2020/csv/ene-2020-12-nde.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2020/csv/ene-2020-11-ond.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2020/csv/ene-2020-10-son.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2020/csv/ene-2020-09-aso.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2020/csv/ene-2020-08-jas.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2020/csv/ene-2020-07-jja.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2020/csv/ene-2020-06-mjj.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2020/csv/ene-2020-05-amj.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2020/csv/ene-2020-04-mam.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2020/csv/ene-2020-03-fma.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2020/csv/ene-2020-02-efm.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2020/csv/ene-2020-01-def.csv",

    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2019/csv/ene-2019-12-nde.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2019/csv/ene-2019-11-ond.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2019/csv/ene-2019-10-son.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2019/csv/ene-2019-09-aso.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2019/csv/ene-2019-08-jas.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2019/csv/ene-2019-07-jja.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2019/csv/ene-2019-06-mjj.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2019/csv/ene-2019-05-amj.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2019/csv/ene-2019-04-mam.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2019/csv/ene-2019-03-fma.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2019/csv/ene-2019-02-efm.csv",
    "https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2019/csv/ene-2019-01-def.csv",
    # Add more URLs as needed
]


In [8]:
delimiter = ";"


In [9]:
# Function to preprocess the "fact_cal" field
def preprocess_fact_cal(df):
    if 'fact_cal' in df.columns:
        df['fact_cal'] = df['fact_cal'].str.replace(',', '.').astype(float)
    return df



In [10]:
# Dictionary to store DataFrames for each trimester
trimester_data = defaultdict(pd.DataFrame)


In [11]:
# Download files and group by trimester
for url in file_urls:
    # Extract filename from URL
    filename = url.split('/')[-1]
    csv_path = os.path.join(source_dir, filename)
    
    print(f"Next {url}")
    # Check if the file already exists
    if not os.path.exists(csv_path):
        # Download the CSV file if it doesn't exist
        download_file(url, csv_path)
        print(f"Downloaded {filename}")
    else:
        print(f"{filename} already exists. Skipping download.")
    
    # Read the CSV file
    df = read_csv_file(csv_path, delimiter=delimiter)
    
    # Preprocess the "fact_cal" field
    df = preprocess_fact_cal(df)
    
    # Extract trimester info (e.g., "04-mam" from "ene-2024-04-mam.csv")
    trimester = '-'.join(filename.split('-')[2:4]).split('.')[0]
    
    # Append data to the corresponding trimester DataFrame
    trimester_data[trimester] = pd.concat([trimester_data[trimester], df], ignore_index=True)
    
    print(f"Added {filename} to the trimester {trimester} DataFrame.")

Next https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2024/csv/ene-2024-04-mam.csv
ene-2024-04-mam.csv already exists. Skipping download.
Detected encoding for /Users/ernestolaval/Documents/Github Repositories/data_chile/data/source/ine/empleo/ene-2024-04-mam.csv: utf-8
Successfully read file with encoding utf-8
Added ene-2024-04-mam.csv to the trimester 04-mam DataFrame.
Next https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2024/csv/ene-2024-03-fma.csv
ene-2024-03-fma.csv already exists. Skipping download.
Detected encoding for /Users/ernestolaval/Documents/Github Repositories/data_chile/data/source/ine/empleo/ene-2024-03-fma.csv: utf-8
Successfully read file with encoding utf-8
Added ene-2024-03-fma.csv to the trimester 03-fma DataFrame.
Next https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2024/csv/ene-2024-02-efm.csv
ene-2024-02-efm.csv already exists. Skipping download.
Detected encoding for /Users/ernestolava

Successfully read file with encoding utf-8
Added ene-2022-08-jas.csv to the trimester 08-jas DataFrame.
Next https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2022/csv/ene-2022-07-jja.csv
ene-2022-07-jja.csv already exists. Skipping download.
Detected encoding for /Users/ernestolaval/Documents/Github Repositories/data_chile/data/source/ine/empleo/ene-2022-07-jja.csv: utf-8
Successfully read file with encoding utf-8
Added ene-2022-07-jja.csv to the trimester 07-jja DataFrame.
Next https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2022/csv/ene-2022-06-mjj.csv
ene-2022-06-mjj.csv already exists. Skipping download.
Detected encoding for /Users/ernestolaval/Documents/Github Repositories/data_chile/data/source/ine/empleo/ene-2022-06-mjj.csv: utf-8
Successfully read file with encoding utf-8
Added ene-2022-06-mjj.csv to the trimester 06-mjj DataFrame.
Next https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2022/csv/ene-2022-05

Successfully read file with encoding ISO-8859-1
Added ene-2021-03-fma.csv to the trimester 03-fma DataFrame.
Next https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2021/csv/ene-2021-02-efm.csv
ene-2021-02-efm.csv already exists. Skipping download.
Detected encoding for /Users/ernestolaval/Documents/Github Repositories/data_chile/data/source/ine/empleo/ene-2021-02-efm.csv: utf-8
Error reading CSV file /Users/ernestolaval/Documents/Github Repositories/data_chile/data/source/ine/empleo/ene-2021-02-efm.csv with encoding utf-8, trying ISO-8859-1
Successfully read file with encoding ISO-8859-1
Added ene-2021-02-efm.csv to the trimester 02-efm DataFrame.
Next https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2021/csv/ene-2021-01-def.csv
ene-2021-01-def.csv already exists. Skipping download.
Detected encoding for /Users/ernestolaval/Documents/Github Repositories/data_chile/data/source/ine/empleo/ene-2021-01-def.csv: utf-8
Error reading CSV file /Us

Added ene-2019-10-son.csv to the trimester 10-son DataFrame.
Next https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2019/csv/ene-2019-09-aso.csv
ene-2019-09-aso.csv already exists. Skipping download.
Detected encoding for /Users/ernestolaval/Documents/Github Repositories/data_chile/data/source/ine/empleo/ene-2019-09-aso.csv: utf-8
Error reading CSV file /Users/ernestolaval/Documents/Github Repositories/data_chile/data/source/ine/empleo/ene-2019-09-aso.csv with encoding utf-8, trying ISO-8859-1
Successfully read file with encoding ISO-8859-1
Added ene-2019-09-aso.csv to the trimester 09-aso DataFrame.
Next https://www.ine.gob.cl/docs/default-source/ocupacion-y-desocupacion/bbdd/2019/csv/ene-2019-08-jas.csv
ene-2019-08-jas.csv already exists. Skipping download.
Detected encoding for /Users/ernestolaval/Documents/Github Repositories/data_chile/data/source/ine/empleo/ene-2019-08-jas.csv: utf-8
Error reading CSV file /Users/ernestolaval/Documents/Github Repositories/d

In [68]:
# List of columns to include
columns_to_include = [
    'ano_trimestre', 
    'mes_central', 
    'ano_encuesta',
    'mes_encuesta',
    
    'region',
    'edad',
    'tramo_edad',
    'sexo',
    'nivel',
    'termino_nivel',
    'cine',
    'nacionalidad',
    
    'b1',
    'b13_rev4cl_caenes', 
    'b14_rev4cl_caenes',
    'r_p_rev4cl_caenes',
    
    'habituales',
    'c10',
    'c11',
    
    'e4',
    
    'efectivas',
    
    'activ',
    'cae_general', 
    'cae_especifico', 
    'categoria_ocupacion',
    'ocup_form',
    'sector',
    'obe',
    'tpi',
    'id',
    'ftp',
    
    'fact_cal'
    
]  # Add other columns as needed



In [69]:
# Save each trimester DataFrame as a Parquet file
for trimester, df in trimester_data.items():
    # Select only the specified columns
    df = df[columns_to_include]
    
    processed_path = os.path.join(processed_dir, f"ene-{trimester}.parquet")
    df.to_parquet(processed_path)
    print(f"Merged DataFrame for trimester {trimester} saved to {processed_path}")

Merged DataFrame for trimester 04-mam saved to /Users/ernestolaval/Documents/Github Repositories/data_chile/data/processed/ine/empleo/ene-04-mam.parquet
Merged DataFrame for trimester 03-fma saved to /Users/ernestolaval/Documents/Github Repositories/data_chile/data/processed/ine/empleo/ene-03-fma.parquet
Merged DataFrame for trimester 02-efm saved to /Users/ernestolaval/Documents/Github Repositories/data_chile/data/processed/ine/empleo/ene-02-efm.parquet
Merged DataFrame for trimester 01-def saved to /Users/ernestolaval/Documents/Github Repositories/data_chile/data/processed/ine/empleo/ene-01-def.parquet
Merged DataFrame for trimester 12-nde saved to /Users/ernestolaval/Documents/Github Repositories/data_chile/data/processed/ine/empleo/ene-12-nde.parquet
Merged DataFrame for trimester 11-ond saved to /Users/ernestolaval/Documents/Github Repositories/data_chile/data/processed/ine/empleo/ene-11-ond.parquet
Merged DataFrame for trimester 10-son saved to /Users/ernestolaval/Documents/Githu

## Regenerar archivo para todos los meses / años con campos claves

In [156]:
import pandas as pd
import numpy as np
import glob

# Define the pattern to match all relevant Parquet files
file_pattern = processed_dir + '/ene-*-*.parquet'
print(file_pattern)

# Use glob to find all files matching the pattern
parquet_files = glob.glob(file_pattern)

# Initialize an empty list to hold the DataFrames
dfs = []

# Iterate over the list of files and read each one into a DataFrame
for file in parquet_files:
    df = pd.read_parquet(file)
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Display the first few rows of the combined DataFrame to check the data
print("Combined DataFrame:")
print(combined_df.head())

# Inspect the distribution of 'cae_especifico' values
print("Distribution of 'cae_especifico' values:")
print(combined_df['cae_especifico'].value_counts())

# Categorize 'Ocupacion' based on 'cae_especifico'
combined_df['Ocupacion'] = combined_df['cae_especifico'].apply(lambda x: 'Ocupados' if 1 <= x <= 7 else ('Desocupados' if 8 <= x <= 9 else 'Unknown'))

# Inspect the distribution of the new 'Ocupacion' column
print("Distribution of 'Ocupacion' values:")
print(combined_df['Ocupacion'].value_counts())

# Categorize 'Nacionalidad'
combined_df['Nacionalidad'] = combined_df['nacionalidad'].apply(lambda x: 'Chilena' if x == 152 else 'Extranjeros')

# Add 'edad_de_trabajar' column
combined_df['edad_de_trabajar'] = combined_df['edad'].apply(lambda x: 1 if x >= 15 else 0)

# Fill null values in the 'sexo' column
combined_df['sexo'] = combined_df['sexo'].fillna('Unknown').astype(str)

# Specify the key dimension fields to group by, including 'edad_de_trabajar' and 'sexo'
key_dimension_fields = [
    'ano_trimestre', 'mes_central', 'Ocupacion', 'Nacionalidad', 'categoria_ocupacion', 'b14_rev4cl_caenes', 'b1', 'ocup_form', 'sector', 'edad_de_trabajar', 'sexo'
]

# Fill null values in the key dimension fields with appropriate placeholders
combined_df['categoria_ocupacion'] = combined_df['categoria_ocupacion'].fillna(-1).astype(int)
combined_df['b14_rev4cl_caenes'] = combined_df['b14_rev4cl_caenes'].fillna(-1).astype(int)
combined_df['b1'] = combined_df['b1'].fillna(-1).astype(int)
combined_df['ocup_form'] = combined_df['ocup_form'].fillna(-1).astype(int)
combined_df['sector'] = combined_df['sector'].fillna(-1).astype(int)

# Aggregate the data by summing the 'fact_cal' metric
aggregated_df = combined_df.groupby(key_dimension_fields)['fact_cal'].sum().reset_index()

# Replace placeholders with np.nan after aggregation for integer columns
aggregated_df['categoria_ocupacion'] = aggregated_df['categoria_ocupacion'].replace(-1, np.nan).astype('Int64')
aggregated_df['b14_rev4cl_caenes'] = aggregated_df['b14_rev4cl_caenes'].replace(-1, np.nan).astype('Int64')
aggregated_df['b1'] = aggregated_df['b1'].replace(-1, np.nan).astype('Int64')
aggregated_df['ocup_form'] = aggregated_df['ocup_form'].replace(-1, np.nan).astype('Int64')
aggregated_df['sector'] = aggregated_df['sector'].replace(-1, np.nan).astype('Int64')

# Display the first few rows of the aggregated DataFrame to check the results
print("Aggregated DataFrame:")
print(aggregated_df.head())

# Save the aggregated DataFrame to a new Parquet file
output_parquet_file = processed_dir + '/ene_sintetica.parquet'
aggregated_df.to_parquet(output_parquet_file, index=False)

print(f"Aggregated data saved to {output_parquet_file}")


/Users/ernestolaval/Documents/Github Repositories/data_chile/data/processed/ine/empleo/ene-*-*.parquet
Combined DataFrame:
   ano_trimestre  mes_central  ano_encuesta  mes_encuesta  region  edad  \
0           2023            6          2023             5      13     9   
1           2023            6          2023             5      13     4   
2           2023            6          2023             5      13    38   
3           2023            6          2023             5      13    43   
4           2023            6          2023             5      13    29   

   tramo_edad  sexo  nivel  termino_nivel  ...  cae_general  cae_especifico  \
0         NaN     2      3              2  ...            0               0   
1         NaN     2      1              1  ...            0               0   
2         5.0     1      8              1  ...            1               1   
3         6.0     2      8              1  ...            9              13   
4         3.0     1      9     

In [13]:
trimester_data

defaultdict(pandas.core.frame.DataFrame,
            {'04-mam':         ano_trimestre  mes_central  ano_encuesta  mes_encuesta  region  \
             0                2024            4          2024             3       3   
             1                2024            4          2024             3       3   
             2                2024            4          2024             3       3   
             3                2024            4          2024             3       3   
             4                2024            4          2024             3       3   
             ...               ...          ...           ...           ...     ...   
             572695           2019            4          2019             5       6   
             572696           2019            4          2019             5       6   
             572697           2019            4          2019             5       9   
             572698           2019            4          2019             5    

In [16]:
# Initialize an empty DataFrame to store the IDs
all_ids = pd.DataFrame()

# Loop through each DataFrame in the dictionary
for key, df in trimester_data.items():
    # Extract the relevant columns
    ids = df[['id_identificacion', 'idrph']].copy()
    
    # Add a column for the source key to track where the IDs came from
    ids['source_key'] = key
    
    # Append to the all_ids DataFrame
    all_ids = pd.concat([all_ids, ids])

# Check for duplicate ids
duplicates = all_ids[all_ids.duplicated(subset=['id_identificacion', 'idrph'], keep=False)]


In [21]:
# Check for duplicate ids
duplicates = all_ids[all_ids.duplicated(subset=['id_identificacion', 'idrph'], keep=False)]

if not duplicates.empty:
    print("Duplicate IDs found:")
    print(duplicates)
else:
    print("No duplicate IDs found.")


Duplicate IDs found:
        id_identificacion        idrph source_key
0                142976.0  11157359096     04-mam
1                142976.0  15665751893     04-mam
2                142977.0   5225326835     04-mam
3                142977.0  17833577210     04-mam
4                142977.0   4027374380     04-mam
...                   ...          ...        ...
472768           267851.0      1392037     05-amj
472769           297633.0      1392048     05-amj
472770           297633.0      1392050     05-amj
472771           289011.0      1392052     05-amj
472772           289011.0      1392053     05-amj

[5978606 rows x 3 columns]


In [22]:
if not duplicates.empty:
    print("Duplicate IDs found:")
    
    # Group by ID and count the number of unique source_keys for each ID
    duplicates_summary = duplicates.groupby(['id_identificacion', 'idrph']).agg({
        'source_key': pd.Series.nunique
    }).reset_index()
    
    # Rename the columns for clarity
    duplicates_summary.columns = ['id_identificacion', 'idrph', 'trimesters_count']
    
    # Get the count of IDs that are duplicated in multiple trimesters
    summary_count = duplicates_summary['trimesters_count'].value_counts().reset_index()
    summary_count.columns = ['trimesters_count', 'id_count']
    
    print("Summary of duplicate IDs across trimesters:")
    print(summary_count)
else:
    print("No duplicate IDs found.")

Duplicate IDs found:
Summary of duplicate IDs across trimesters:
    trimesters_count  id_count
0                  3    312347
1                 12    141989
2                  6     98082
3                  9     95492
4                  2     60522
5                 11     32808
6                  4     25962
7                  5     24711
8                  7     22377
9                  8     21348
10                10     10062


ERROR: Could not find a version that satisfies the requirement ace_tools (from versions: none)
ERROR: No matching distribution found for ace_tools
Note: you may need to restart the kernel to use updated packages.


In [25]:
df1 = trimester_data['03-fma']

In [26]:
df2 = trimester_data['04-mam']

In [28]:
df1 = df1[df1['ano_trimestre'] == 2024]
df2 = df2[df2['ano_trimestre'] == 2024]
 

In [31]:
    # Extract the overlapping months
    overlapping_months = set(df1['mes_encuesta']).intersection(set(df2['mes_encuesta']))


In [32]:
overlapping_months

{3, 4}

In [33]:
    df1_overlap = df1[df1['mes_encuesta'].isin(overlapping_months)]
    df2_overlap = df2[df2['mes_encuesta'].isin(overlapping_months)]


In [35]:
# Compare the IDs for these overlapping months
common_ids_df1 = df1_overlap['id_identificacion'].unique()
common_ids_df2 = df2_overlap['id_identificacion'].unique()
 

In [36]:
common_ids = set(common_ids_df1).intersection(set(common_ids_df2))

In [39]:
print(f"Overlapping months between  and for the year {2024}: {overlapping_months}")
print(f"Number of common IDs in overlapping months: {len(common_ids)}")
print(f"Common IDs: {common_ids}")

Overlapping months between  and for the year 2024: {3, 4}
Number of common IDs in overlapping months: 24195
Common IDs: {142976, 142977, 142978, 142979, 142980, 142981, 142982, 142984, 142985, 142986, 142987, 142989, 142992, 142994, 142995, 142996, 142997, 142998, 142999, 143000, 143003, 143005, 143006, 143008, 143009, 143011, 143012, 143013, 143015, 143018, 143021, 143022, 143023, 143024, 143025, 143026, 143027, 143028, 143029, 143030, 143031, 143032, 143033, 143034, 143035, 143037, 143038, 143039, 143041, 143042, 143043, 143046, 143047, 143048, 143049, 143051, 143052, 143055, 143057, 143058, 143061, 143062, 143063, 143064, 143065, 143066, 143067, 143068, 143070, 143071, 143072, 143073, 143074, 143076, 143079, 143080, 143081, 143082, 143083, 143084, 143085, 143086, 143087, 143088, 143089, 143090, 143091, 143092, 143095, 143096, 143098, 143099, 143101, 143102, 143103, 143105, 143106, 143110, 143111, 143112, 143113, 143114, 143115, 143116, 143117, 143118, 143119, 143121, 143123, 143124,

In [30]:
    # Extract the overlapping months
    overlapping_months = set(df1['mes_encuesta']).intersection(set(df2['mes_encuesta']))
    
    if not overlapping_months:
        print(f"No overlapping months found between {trimester1} and {trimester2} for the year {year}.")

    # Filter the DataFrames for the overlapping months
    df1_overlap = df1[df1['mes_encuesta'].isin(overlapping_months)]
    df2_overlap = df2[df2['mes_encuesta'].isin(overlapping_months)]
    
    # Compare the IDs for these overlapping months
    common_ids_df1 = df1_overlap['id_identificacion'].unique()
    common_ids_df2 = df2_overlap['id_identificacion'].unique()
    
    common_ids = set(common_ids_df1).intersection(set(common_ids_df2))
    
    print(f"Overlapping months between {trimester1} and {trimester2} for the year {year}: {overlapping_months}")
    print(f"Number of common IDs in overlapping months: {len(common_ids)}")
    print(f"Common IDs: {common_ids}")

NameError: name 'trimester1' is not defined

In [ ]:
import pandas as pd

# Assuming 'trimester_data' is your dictionary of DataFrames
# Example:
# trimester_data = {
#     '3-4-5': df1,
#     '4-5-6': df2,
#     ...
# }

def find_overlapping_ids_for_year(trimester_data, trimester1, trimester2, year):
    # Extract the DataFrames for the specified trimesters
    df1 = trimester_data[trimester1]
    df2 = trimester_data[trimester2]
    
    # Filter the DataFrames for the specified year
    df1 = df1[df1['ano_trimestre'] == year]
    df2 = df2[df2['ano_trimestre'] == year]
    
    # Extract the overlapping months
    overlapping_months = set(df1['mes_encuesta']).intersection(set(df2['mes_encuesta']))
    
    if not overlapping_months:
        print(f"No overlapping months found between {trimester1} and {trimester2} for the year {year}.")
        return None

    # Filter the DataFrames for the overlapping months
    df1_overlap = df1[df1['mes_encuesta'].isin(overlapping_months)]
    df2_overlap = df2[df2['mes_encuesta'].isin(overlapping_months)]
    
    # Compare the IDs for these overlapping months
    common_ids_df1 = df1_overlap['id_identificacion'].unique()
    common_ids_df2 = df2_overlap['id_identificacion'].unique()
    
    common_ids = set(common_ids_df1).intersection(set(common_ids_df2))
    
    print(f"Overlapping months between {trimester1} and {trimester2} for the year {year}: {overlapping_months}")
    print(f"Number of common IDs in overlapping months: {len(common_ids)}")
    print(f"Common IDs: {common_ids}")
    
    return common_ids

# Example usage
year_to_check = 2023
overlapping_ids = find_overlapping_ids_for_year(trimester_data, '3-4-5', '4-5-6', year_to_check)

# If you want to check for multiple pairs of trimesters for a specific year
def check_multiple_pairs_for_year(trimester_data, pairs, year):
    results = {}
    for t1, t2 in pairs:
        common_ids = find_overlapping_ids_for_year(trimester_data, t1, t2, year)
        if common_ids is not None:
            results[(t1, t2)] = common_ids
    return results

# Example pairs to check
pairs_to_check = [('3-4-5', '4-5-6'), ('2-3-4', '3-4-5')]

# Get the results for multiple pairs
multiple_results = check_multiple_pairs_for_year(trimester_data, pairs_to_check, year_to_check)
